In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict,LeaveOneOut,KFold
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn import preprocessing
from sklearn.metrics import make_scorer
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection
from scipy.stats import sem
import subprocess

In [7]:
def eval_metrics(y_true,y_pred,prob):
    print('Accuracy:', round(metrics.accuracy_score(y_true,y_pred),3))
    print('Precision:', round(metrics.precision_score(y_true, y_pred),3))
    print('Sensitivity:', round(metrics.recall_score(y_true,y_pred),3))
    temp_tn, temp_fp, temp_fn, temp_tp = metrics.confusion_matrix(y_true,y_pred).ravel()
    temp_specificity = temp_tn / (temp_tn + temp_fp)
    print('Specificity:', round(temp_specificity,3))
    print('F1-score:',round(metrics.f1_score(y_true,y_pred),3))
    print('AUC: ', round(metrics.roc_auc_score(y_true, prob),3))
    print('MCC: ', round(metrics.matthews_corrcoef(y_true,y_pred),3))

## Build features for independent test dataset

In [19]:
def buildFeatures(ampfile, nonampfile ,out, split1_prop, split2_prop):
    '''
    Function to build required features by calling on the R script 
        
    Parameters:
      ampfile: Path to a fasta format AMP file, or a text file in fasta format
      non-ampfile: Path to a fasta format Non-AMP file, or a text file in fasta format
      out: Name of output file
      split1_prop: First split (left-side) fraction of the peptide sequence 
      split2_prop: Second split (middle) fraction of the peptide sequence
      
      *split1/2_prop must be less than 1
    Returns:
      Write the feature matrix to the 'features' folder
    '''
    
    # Command to execute
    command = f"Rscript buildFeatures.R {ampfile} {nonampfile} \
                    {out} {split1_prop} {split2_prop}"
    process = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    # Print output
    for line in process.stdout:
        print(line.decode("utf-8"),end='')
    # Wait for the command to finish
    process.wait()

In [20]:
## build the feature matrix for the training data 
buildFeatures('Training_Dataset/iAMPpred_pos_fasta.txt','Training_Dataset/iAMPpred_neg_fasta.txt',\
             'feature_matrix/iAMPpred_features.csv',0.2,0.6)

Preprocessing done!
#########AMP-Sequence Summary############
984 peptide sequences are found after preprocessing!
Minimum sequence length is 10 
Maximum sequence length is 255 
Average sequence length is 44.5437 
##########################################
#########NonAMP-Sequence Summary############
984 peptide sequences are found after preprocessing!
Minimum sequence length is 10 
Maximum sequence length is 94 
Average sequence length is 29.9685 
##########################################
AAC done!
PHYC done!
PAAC done!
NAAC done!
SAAC done!
Finished!
Feature matrix stored in feature_matrix/iAMPpred_features.csv 


In [21]:
## build the feature matrix for the independent testing data
buildFeatures('Independent_set/Independent_Dataset/human_Positive.txt','Independent_set/Independent_Dataset/human_Negative.txt',
             'feature_matrix/dbAMP_human_features.csv',0.2,0.6)

Preprocessing done!
#########AMP-Sequence Summary############
39 peptide sequences are found after preprocessing!
Minimum sequence length is 12 
Maximum sequence length is 456 
Average sequence length is 78.07692 
##########################################
#########NonAMP-Sequence Summary############
894 peptide sequences are found after preprocessing!
Minimum sequence length is 52 
Maximum sequence length is 500 
Average sequence length is 284.868 
##########################################
AAC done!
PHYC done!
PAAC done!
NAAC done!
SAAC done!
Finished!
Feature matrix stored in feature_matrix/dbAMP_human_features.csv 


In [2]:
## Prepare independent testing data
X_test = pd.read_csv('feature_matrix/dbAMP_human_features.csv',index_col=0)
y_test = X_test.pop('labels')

## Prepare training data
X_train = pd.read_csv('feature_matrix/iAMPpred_features.csv',index_col=0)
y_train = X_train.pop('labels')

## Random Projection based model training

In [23]:
def score_ensemble(scores,y_tr,dim,Runs):
    '''
    Function to ensemble scores across RP runs 
        
    Parameters:
      scores: prediction score matrix
      y_tr: actual labels
      Runs: number of RP runs
      
    Returns:
      None. Print out the results
    '''
    # average the score across all runs of RP
    predictions_res = pd.DataFrame(np.mean(scores,axis=1))

    # score > 0 will be 1, < 0 will be 0
    y_pred = (predictions_res[0]>0).astype(int)
    
    # evaluation metrics
    acc = metrics.accuracy_score(y_tr,y_pred)
    recall = metrics.recall_score(y_tr,y_pred) 
    tn, fp, fn, tp = metrics.confusion_matrix(y_tr, y_pred).ravel()
    specificity = tn / (tn+fp)
    precision = metrics.precision_score(y_tr, y_pred)
    auc = metrics.roc_auc_score(y_tr, predictions_res[0])
    f1 = metrics.f1_score(y_tr,y_pred)
    
    # below is to find the std across all runs
    temp_acc = []
    temp_sn = []
    temp_sp = []
    temp_auc = []
    temp_prec = []
    temp_f1 = []
    
    # loop through all runs and calculate metrics for each run
    for i in range(Runs):
        y_pred_perrun = (scores[f'Run_{i}'] >0).astype(int)

        temp_tn, temp_fp, temp_fn, temp_tp = \
            metrics.confusion_matrix(y_tr, y_pred_perrun).ravel()
        
        temp_acc.append(metrics.accuracy_score(y_tr,y_pred_perrun))
        temp_sn.append(metrics.recall_score(y_tr,y_pred_perrun))
        temp_sp.append(temp_tn / (temp_tn + temp_fp))
        temp_auc.append(metrics.roc_auc_score(y_tr, scores[f'Run_{i}']))
        temp_prec.append(metrics.precision_score(y_tr, y_pred_perrun))
        temp_f1.append(metrics.f1_score(y_tr,y_pred_perrun))
        
    std_acc = np.std(temp_acc)
    std_sn = np.std(temp_sn)
    std_sp = np.std(temp_sp)
    std_auc = np.std(temp_auc)
    std_prec = np.std(temp_prec)
    std_f1 = np.std(temp_f1)
    
    print(f"Dimension {dim} =>> \n","Accuracy : %.2f" % (acc*100),'±',"%.3f \n" % std_acc,
          " Sensitivity : %.2f" % (recall*100),'±',"%.3f \n" % std_sn, 
         " Specificity : %.2f" % (specificity*100),'±',"%.3f \n" % std_sp,
         " Precision : %.2f" % (precision*100),'±',"%.3f \n" % std_prec,
         " AUC : %.2f" % (auc*100),'±',"%.3f \n" % std_auc,
         " F1 : %.2f" % (f1*100),'±',"%.3f \n" % std_f1)

In [24]:
def RP_train(X,y,dim,method='Gaussian',Runs=10):    
    '''
    Function to train an SVM classifier with \
        random projection-based dimensionality reduction
        
    Parameters:
      X: Input feature matrix for training the model
      y: AMP/Non_AMP labels for training the model
      dim: Desired dimensionality of the projected space
      method: Random projection method to use (default: 'Gaussian')
      Runs: Number of runs to perform RP (default: 10)
    Returns:
      None (prints evaluation metrics)
    '''
    scores = pd.DataFrame()
    for i in range(Runs):
        
        if method == 'Gaussian':
            transformer = \
            GaussianRandomProjection(random_state=i,n_components=dim)
        elif method == 'Sparse':
            transformer = \
            SparseRandomProjection(random_state=i, n_components=dim)
        
        Xtr = transformer.fit_transform(X)
        ytr = np.array(y).ravel()

        ## Define model
        svm_model = SVC(random_state=1)
        ## Define the parameter grid for grid search
        param_grid = {'C': [0.1, 1, 10, 100], 
                      'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                      'kernel': ['rbf']} 

        grid_search = GridSearchCV(svm_model, param_grid, \
                                   cv=10,n_jobs=-1,scoring='accuracy')
        grid_search.fit(Xtr, ytr)
        
        # define a new classifier with the best hyperparameters
        clf = SVC(**grid_search.best_params_,random_state=1) 

        # use kfold CV to evaluate model performance 
        kfold = KFold(n_splits=10, shuffle=True,random_state=i)
        
        # predict each class with a score using CV
        score = cross_val_predict(clf, Xtr, ytr, \
                                  cv=kfold, method='decision_function')
        # store the score for each run of RP
        score = pd.DataFrame(score)
        score.columns = [f'Run_{i}']
        scores = pd.concat([scores,score],axis=1)
    
    score_ensemble(scores,ytr,dim,Runs)

## Model evaluation on training data

In [25]:
for i in [50,100,150]:
    RP_train(X_train,y_train,dim = i)

Dimension 50 =>> 
 Accuracy : 93.09 ± 0.005 
  Sensitivity : 90.55 ± 0.004 
  Specificity : 95.63 ± 0.011 
  Precision : 95.40 ± 0.010 
  AUC : 97.63 ± 0.002 
  F1 : 92.91 ± 0.005 

Dimension 100 =>> 
 Accuracy : 93.60 ± 0.005 
  Sensitivity : 91.97 ± 0.007 
  Specificity : 95.22 ± 0.007 
  Precision : 95.06 ± 0.007 
  AUC : 97.87 ± 0.002 
  F1 : 93.49 ± 0.005 

Dimension 150 =>> 
 Accuracy : 93.65 ± 0.004 
  Sensitivity : 91.97 ± 0.007 
  Specificity : 95.33 ± 0.005 
  Precision : 95.16 ± 0.005 
  AUC : 97.82 ± 0.002 
  F1 : 93.54 ± 0.004 



## Random Projection based model testing on independent testing data

In [26]:
## Random Projection for independent test
def RP_indTest(trainX,trainy,testX,testy,dim,method='Gaussian',Runs=10):
    '''
    Similar to the code above
    '''
    scores = pd.DataFrame()
    for i in range(Runs):
        ## RP
        if method == 'Gaussian':
            transformer = GaussianRandomProjection(random_state=i,n_components=dim)
        elif method == 'Sparse':
            transformer = SparseRandomProjection(random_state=i, n_components=dim)
        
        Xtt = transformer.fit_transform(testX)
        ytt = np.array(testy).ravel()
        
        Xtr = transformer.fit_transform(trainX)
        ytr = np.array(trainy).ravel()
        
        ## Define model
        svm_model = SVC(random_state=1)
        # Define the parameter grid for grid search
        param_grid = {'C': [0.1, 1, 10, 100], 
                      'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                      'kernel': ['rbf']} 

        grid_search = GridSearchCV(svm_model, param_grid, cv=10,n_jobs=-1,scoring='accuracy')
        grid_search.fit(Xtr, ytr)
        
        # use best hyperparameters
        clf = SVC(**grid_search.best_params_,random_state=1) 
        
        clf.fit(Xtr,ytr)
        # predict each class with a score
        score = clf.decision_function(Xtt)## predict on independent test data
        score = pd.DataFrame(score)
        score.columns = [f'Run_{i}']
        scores = pd.concat([scores,score],axis=1)
    
    score_ensemble(scores,ytt,dim,Runs)

## dbAMP Human Dataset Results

In [27]:
for i in [50,100,150]:
    RP_indTest(X_train,y_train,X_test,y_test,i,Runs=5)

Dimension 50 =>> 
 Accuracy : 66.13 ± 0.014 
  Sensitivity : 87.18 ± 0.028 
  Specificity : 65.21 ± 0.015 
  Precision : 9.86 ± 0.005 
  AUC : 70.70 ± 0.027 
  F1 : 17.71 ± 0.008 

Dimension 100 =>> 
 Accuracy : 66.88 ± 0.016 
  Sensitivity : 87.18 ± 0.062 
  Specificity : 66.00 ± 0.017 
  Precision : 10.06 ± 0.007 
  AUC : 72.03 ± 0.027 
  F1 : 18.04 ± 0.013 

Dimension 150 =>> 
 Accuracy : 66.13 ± 0.014 
  Sensitivity : 87.18 ± 0.035 
  Specificity : 65.21 ± 0.015 
  Precision : 9.86 ± 0.005 
  AUC : 71.77 ± 0.017 
  F1 : 17.71 ± 0.008 



## SVM-based model training 

In [3]:
def model(X,y,feature_names):
    Xdt = np.array(X) 
    ydt = np.array(y).ravel()

    ## Use all data for grid search
    svm_model = SVC()

    # Define the parameter grid for grid search
    param_grid = {'C': [0.1, 1, 10, 100, 1000], 
                  'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                  'kernel': ['rbf','sigmoid']} 

    # Perform grid search with 10 cross-validation
    grid_search = GridSearchCV(svm_model, param_grid, cv=10,n_jobs=-1,scoring='accuracy')

    # Fit the model with grid search
    grid_search.fit(Xdt, ydt)
    print(f'Training on {feature_names}!')
    print(f"Best parameters found: ", grid_search.best_params_)
    print(f"Best score found: ", grid_search.best_score_)
    
    ## Repeat 10 fold cv for 10 times
    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=1)
    
    # use best hyperparameters
    clf = SVC(**grid_search.best_params_,random_state=1) 
    
    # evaluation metrics and print them out
    accuracy_scores = cross_val_score(clf , Xdt, ydt, scoring='accuracy', cv=rkf, n_jobs=-1)
    f1_scores = cross_val_score(clf , Xdt, ydt, scoring='f1', cv=rkf, n_jobs=-1)
    precision = cross_val_score(clf , Xdt, ydt, scoring='precision', cv=rkf, n_jobs=-1)
    recall = cross_val_score(clf , Xdt, ydt, scoring='recall', cv=rkf, n_jobs=-1)

    specificity = make_scorer(recall_score, pos_label=0)
    specificity = cross_val_score(clf, Xdt, ydt, cv=rkf, scoring = specificity,n_jobs=-1)
    
    auc = cross_val_score(clf, Xdt, ydt, cv=rkf, scoring = 'roc_auc',n_jobs=-1)
    
    print(f'Accuracy:', round(accuracy_scores.mean()*100,2),'±'
          ,round(sem(accuracy_scores*100),3))
    print(f'F1-scores:', round(f1_scores.mean()*100,2),'±'
          ,round(sem(f1_scores*100),3))
    print(f'Precision:', round(precision.mean()*100,2),'±'
          ,round(sem(precision*100),3))
    print(f'Sensitivity:', round(recall.mean()*100,2),'±'
          ,round(sem(recall*100),3))
    print(f'Specificity:', round(specificity.mean()*100,2),'±'
          ,round(sem(specificity*100),3))
    print(f'{feature_names} AUC:', round(auc.mean()*100,2),'±'
          ,round(sem(auc*100),3))
    print(f'Training done!')
    print('########################')
    
    return clf

## iAMPpred 

## Training on iAMPpred training data

In [4]:
clf = model(X_train,y_train,'all features')

Training on all features!
Best parameters found:  {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
Best score found:  0.9329068683310888
Accuracy: 93.33 ± 0.169
F1-scores: 93.18 ± 0.179
Precision: 94.77 ± 0.245
Sensitivity: 91.71 ± 0.278
Specificity: 94.97 ± 0.225
all features AUC: 97.75 ± 0.084
Training done!
########################


## Predicting on independent human dataset

In [8]:
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
prob = clf.decision_function(X_test)
print('Prediction results for human dataset')
eval_metrics(y_test,y_pred,prob)

Prediction results for human dataset
Accuracy: 0.636
Precision: 0.094
Sensitivity: 0.897
Specificity: 0.624
F1-score: 0.171
AUC:  0.711
MCC:  0.213
